In [6]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import subprocess
import pymc3 as pm
import seaborn as sns

In [4]:
import os

os.chdir("../..")
os.getcwd()

'/home/mario/PhD/spatial_lda'

In [2]:
covariates = ["host_is_superhost", "accommodates", "bathrooms", "bedrooms", 
              "number_of_reviews", "review_scores_rating", "instant_bookable"]

In [7]:
with open("data/amsterdam.pkl", "rb") as fp:
    datas = pickle.load(fp)

In [8]:
covariates_by_group = []
for i, cov in enumerate(datas["cov"]):      
    currDf = cov[covariates].copy()
    if(currDf.isna().sum().sum()):
        print("Nans in group {0}".format(i))
    
    currDf["host_is_superhost"] = currDf["host_is_superhost"].apply(lambda x: int(x == "t"))
    currDf["instant_bookable"] = currDf["instant_bookable"].apply(lambda x: int(x == "t"))
    
    curr = currDf.values.astype(np.float)
    covariates_by_group.append(curr)

In [14]:
X = np.vstack(covariates_by_group)
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
Y = np.concatenate(datas["y"])

In [25]:
mu0 = np.zeros(X.shape[1])
lambda0 = np.ones((X.shape[1], X.shape[1])) * 0.01
a = b = 1.0

xtx = np.matmul(X.T, X)
betahat = np.linalg.solve(xtx, np.matmul(X.T, Y))
lambda_n = (xtx + lambda0)
mu_n = np.linalg.solve(
    lambda_n, np.matmul(xtx, betahat) + np.matmul(lambda0, mu0))
an = a + len(Y) / 2
bn = b + 0.5 * (
    np.linalg.norm(Y) ** 2 + 
    np.matmul(mu0.T, np.matmul(lambda0, mu0)) -
    np.matmul(mu_n.T, np.matmul(lambda_n, mu_n))
)

In [58]:
from scipy.stats import invgamma, norm
from scipy.stats import multivariate_normal as mvn

lambda_n_inv = np.linalg.inv(lambda_n)

var_samples = invgamma.rvs(an, scale=bn, size=1000)
beta_samples = np.vstack(
    [mvn.rvs(mu_n, lambda_n_inv * var) for var in var_samples])

In [59]:
mean_samples = np.einsum("nk,rk->nr", X, beta_samples).T

In [67]:
eval_dens = np.zeros_like(mean_samples)
for r in range(mean_samples.shape[0]):
    eval_dens[r, :] = norm.pdf(Y, loc=mean_samples[r, :], scale=np.sqrt(var_samples[r]))

In [68]:
from spatial_mix.utils import *

In [72]:
lpml(eval_dens) - 3000

-101729.59719768849

In [77]:
-1e5 - 1

-100001.0

In [73]:
(waic(eval_dens) - 3000) * (-2)

203459.26709961396